In [1]:
import cv2
import numpy as np
import utlis

In [2]:
def scan_image(imagePath):
    
    heightImg = 640
    widthImg  = 480

    img = cv2.imread(imagePath)
    imgCopy = img.copy() # Keep an original copy of the image
    img = cv2.resize(img, (widthImg, heightImg)) # Resize the image
    imgResizeCopy = img.copy() # Copy of the resize image

    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    green_lo=np.array([10, 25, 25])# Define lower and uppper limits of what we call "green"
    green_hi=np.array([80, 255,255])
    mask=cv2.inRange(hsv,green_lo,green_hi)# Mask image to only select greens
    img[mask>0]=(0,0,0)# Change image to black where we found green

    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert image to grayscale
    imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1) # Add gaussian blur
    imgThreshold = cv2.Canny(imgBlur,200,255) # Appy canny blur
    kernel = np.ones((5, 5))
    imgDial = cv2.dilate(imgThreshold, kernel, iterations=2) # Apply dialation
    imgThreshold = cv2.erode(imgDial, kernel, iterations=1)  # Apply erosion  

    # FIND ALL CONTOURs
    imgContours = img.copy() # Copy image for display purposes
    imgBigContour = img.copy() # Copy image for display purposes
    contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Find all contours
    cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 10) # Draw all detected contours

    # FIND THE BIGGEST CONTOUR AND WRAP THE IMAGE
    biggest, maxArea = utlis.biggestContour(contours) # Find the biggest contour

    if biggest.size != 0:
        biggest=utlis.reorder(biggest)
        cv2.drawContours(imgBigContour, biggest, -1, (0, 255, 0), 20) # Draw the biggest contour
        imgBigContour = utlis.drawRectangle(imgBigContour,biggest,2)
        pts1 = np.float32(biggest) # Prepare point for wrap
        pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # Prepare point for wrap
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))

        #REMOVE 20 PIXELS FORM EACH SIDE
        imgWarpColored=imgWarpColored[20:imgWarpColored.shape[0] - 20, 20:imgWarpColored.shape[1] - 20]
        imgWarpColored = cv2.resize(imgWarpColored,(widthImg,heightImg))
        
        return imgWarpColored
    else:
        return imgResizeCopy
    

In [3]:
def count_caterpillars(scanned_img):

    scannedImage = scanned_img.copy() # Keep an original copy of the image
    grayImage = cv2.cvtColor(scanned_img, cv2.COLOR_BGR2GRAY)#turn to grayscale
    (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 130, 255, cv2.THRESH_BINARY)
    imagem = cv2.bitwise_not(grayImage)#bit-wise conversion of the image    

    # First threshlding image using adaptive thresholding
    thresholded_image = cv2.adaptiveThreshold(imagem, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, -5)
    # Then errode thresholded image
    eroded_image = cv2.erode(thresholded_image.astype('uint8'),None,iterations = 1)

    # You need to choose 4 or 8 for connectivity type
    connectivity = 4  
    # Perform the operation
    output = cv2.connectedComponentsWithStats(eroded_image, connectivity, cv2.CV_32S)
    # Get the results
    # The first cell is the number of labels
    num_labels = output[0]
    #print("Number of total connected components including small particles : " ,num_labels)
    # The second cell is the label matrix 
    labels = output[1]
    # The third cell is the stat matrix
    stats = output[2]
    # The fourth cell is the centroid matrix
    centroids = output[3]

    pixel_areas = []
    # print(stats)
    caterpillar_count = 0

    for i in range(1, num_labels):
        if stats[i][4] > 20:#Area threshold
    #         print(stats[i][4])
            caterpillar_count+=1
            left_x = stats[i][0]
            up_y = stats[i][1]
            right_x = left_x + stats[i][2]
            down_y = up_y + stats[i][3]
            cv2.rectangle(scanned_img,(left_x, up_y),(right_x,down_y),(0,255,0),3)

    cv2.imwrite('Output/calculated.jpg', scanned_img)
    #print("Dust particles : " ,num_labels - caterpillar_count)
    print("Total Number of Caterpillars : " , caterpillar_count)


In [4]:
imagePath = "Images/2.jpg"
scanned_img = scan_image(imagePath)
count_caterpillars(scanned_img)


Total Number of Caterpillars :  25
